## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
from subprocess import Popen, PIPE
import os
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode

client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 2
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [client.add_str(_nickname + 'avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [10_000_000] * NUMBER_OF_PARTICIPANTS

INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(20)
COEF_DOWN = str(5)
TARGET_CLAIM = str(2)

WALLET_ADDRESS = os.getenv('WALLET_ADDRESS')
NODE_URL = 'https://rpc.space-pussy-1.cybernode.ai:443'
NETWORK = 'space-pussy-1'

INIT_SUBSPACE_CONTRACTS = False
SUBSPACE_CODE_ID = str(21)
NAME_SUBSPACE_CONTRACT_ADDRESS = 'bostrom19zmt9dlz3wzu6jqgr9409el7nvxw66u8n67ta5424kwcg0qf269s6t03zt'
AVATAR_SUBSPACE_CONTRACT_ADDRESS = 'bostrom16ulvzd0xp9g6wdhnnhz584ed8xzxm5wkmrsznnzk7qsskegkfutq5djtqs'
PROOF_SUBSPACE_CONTRACT_ADDRESS = 'bostrom1ghwwe2nhxjfy7qulgu6zzmghmnjj3rx8xldr8aytghayxlftz6aqxy4c7k'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(25)
PASSPORT_CONTRACT_ADDRESS = 'bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262'

ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'


def execute_bash(bash_command: str, display_data: bool = False) -> [str, str]:
    _output, _error = Popen(bash_command, shell=True, stdout=PIPE, stderr=PIPE).communicate()
    if _error:
        display_data = True
    if display_data:
        print(bash_command)
        if _output:
            try:
                print(json.dumps(json.loads(_output.decode('utf-8')), indent=4, sort_keys=True))
            except json.JSONDecodeError:
                print(_output)
        if _error:
            print(_error)
    return _output.decode('utf-8'), _error.decode('utf-8')

/Users/sergenedashkovsky/Library/Python/3.8/lib/python/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.10.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
claims_list = []
for i in range(NUMBER_OF_PARTICIPANTS):
    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
                  'nickname': NICKNAME_LIST[i],
                  'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], 'cosmos')
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    # message_json = {
    #     "nickname": claim_item['nickname'],
    #     "gift_claiming_address": claim_item['ethereum_address'],
    #     "gift_amount": claim_item['amount'],
    #     "target_address": claim_item['bostrom_address']
    # }
    # claim_item['message'] = json.dumps(message_json)
    claim_item['message'] = f"{WALLET_ADDRESS}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    # print(signed_message.signature)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix='cosmos')
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    claims_list.append(claim_item)

claims_df = pd.DataFrame(claims_list)
claims_df.to_csv('claims_ethereum_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,10000000,john16455994800,QmdftF8AWKtpqoC3MpNmrapVWiumGTdCPXrvYmz8V17ZVv,0x4608e844331df74ca530e0b8cbb80057a8e8818a,0x7299cd04965e71b84f0e944db69c4fef1e4fe4effcef...,bostrom1za5gkyfz5jp3vhm7lphwgfdvpud87p6m2dj0v9,cosmos1za5gkyfz5jp3vhm7lphwgfdvpud87p6mf7xujz,finger never cheese neck squeeze license dawn ...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0xf533d1b1d62ec166b3a9c5663ee614323800eabecb84...,{'pub_key': 'AppIVn2o22irKU/GUchgMTturtT8BQ4Kt...,eyJwdWJfa2V5IjoiQXBwSVZuMm8yMmlyS1UvR1VjaGdNVH...
1,10000000,john16455994801,Qma6rDtCvYN6UWrUV978rmyZrxtKWWg8buYzy5H8Az3ux3,0x9f791bd90b3b3a8f888a1414fba806d7aba85e97,0x7e461f3a4ea595abdbc47ea4385a1ef6d777175545d1...,bostrom120gnxgzj9hv53meq7h24jkgtfw8w2lh4awp2vp,cosmos120gnxgzj9hv53meq7h24jkgtfw8w2lh47a4ejx,rookie library small humor execute antique tru...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x438ad98a4d25cf3780a67e1a55ade66f04a3ac529b14...,{'pub_key': 'AqhmlaczxtqDaBHBjhhtYS5AycZBqnfRG...,eyJwdWJfa2V5IjoiQXFobWxhY3p4dHFEYUJIQmpoaHRZUz...


## Instantiate Contracts

In [3]:
def instantiate_contract(init_query: str, contract_code_id: str, contract_label: str, amount: str = '', display_data: bool = False) -> str:
    _init_output, _init_error = execute_bash(
        f'''INIT='{init_query}' \
            && cyber tx wasm instantiate {contract_code_id} "$INIT" --from $WALLET {'--amount ' + amount + 'boot' if amount else ''} --label "{contract_label}" \
            -y --gas 3500000 --broadcast-mode block -o json --chain-id={NETWORK} --node={NODE_URL}''')
    if display_data:
        try:
            print(json.dumps(json.loads(_init_output.decode('utf-8')), indent=4, sort_keys=True))
        except json.JSONDecodeError:
            print(_init_output)
    if _init_error:
        print(_init_error)
    _init_json = json.loads(_init_output)
    return [event['attributes'][0]['value'] for event in _init_json['logs'][0]['events'] if event['type'] == 'instantiate'][0]

### Instantiate Subspace Contracts

In [4]:
if INIT_SUBSPACE_CONTRACTS:
    name_subspace_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBSPACE_CODE_ID,
            contract_label='test name subspace')
    avatar_subspace_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBSPACE_CODE_ID,
            contract_label='test avatar subspace')
    proof_subspace_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBSPACE_CODE_ID,
            contract_label='test proof subspace')
else:
    name_subspace_contract_address = NAME_SUBSPACE_CONTRACT_ADDRESS
    avatar_subspace_contract_address = AVATAR_SUBSPACE_CONTRACT_ADDRESS
    proof_subspace_contract_address = PROOF_SUBSPACE_CONTRACT_ADDRESS
print(f'Name subspace contract address: {name_subspace_contract_address}\n'
      f'Avatar subspace contract address: {avatar_subspace_contract_address}\n'
      f'Proof subspace contract address: {proof_subspace_contract_address}')

Name subspace contract address: bostrom19zmt9dlz3wzu6jqgr9409el7nvxw66u8n67ta5424kwcg0qf269s6t03zt
Avatar subspace contract address: bostrom16ulvzd0xp9g6wdhnnhz584ed8xzxm5wkmrsznnzk7qsskegkfutq5djtqs
Proof subspace contract address: bostrom1ghwwe2nhxjfy7qulgu6zzmghmnjj3rx8xldr8aytghayxlftz6aqxy4c7k


### Instantiate Passport Contract

In [5]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subspace": "{avatar_subspace_contract_address}", "name_subspace": "{name_subspace_contract_address}", "proof_subspace": "{proof_subspace_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport')
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra


### Set executor in the Subspace Contracts

In [6]:
def set_executor_subspace(subspace_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_bash(
                f'''REGISTER='{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}' \
                    && CONTRACT="{subspace_contract_address}" \
                    && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=600000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBSPACE_CONTRACTS:
    set_executor_subspace(subspace_contract_address=name_subspace_contract_address, new_executor_address=passport_contract_address)
    set_executor_subspace(subspace_contract_address=avatar_subspace_contract_address, new_executor_address=passport_contract_address)
    set_executor_subspace(subspace_contract_address=proof_subspace_contract_address, new_executor_address=passport_contract_address)

### Instantiate Gift Contract

In [7]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test gift')
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1xj68jgjr6u02njplhmkl2f4gg58uueayfrn99x2png87jpsvwx0sk58jjv


In [8]:
def get_contract_name(contract_address: str) -> str:
    address_dict = {gift_contract_address: 'Gift Contract',
                    passport_contract_address: 'Passport Contract',
                    WALLET_ADDRESS: 'Wallet',
                    name_subspace_contract_address: 'Name Subspace Contract',
                    avatar_subspace_contract_address: 'Avatar Subspace Contract',
                    proof_subspace_contract_address: 'Proof Subspace Contract'}
    try:
        return address_dict[contract_address]
    except KeyError:
        return contract_address


def parse_contract_execution_json(contract_execution_json: str) -> None:
    for log_item in json.loads(contract_execution_json)['logs']:
        for event_item in log_item['events']:
            print('')
            if event_item['type'] == 'message':
                if len(event_item["attributes"]) == 3:
                    print(f'message from {get_contract_name(event_item["attributes"][-1]["value"])} {event_item["attributes"][1]["value"]} {event_item["attributes"][0]["value"]}')
                else:
                    print(event_item)
            elif event_item['type'] == 'execute':
                print('execute')
                for attr_item in event_item["attributes"]:
                    if attr_item["key"] == '_contract_address':
                        print(f'\texecute contract: {get_contract_name(attr_item["value"])}')
                    else:
                        print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'reply':
                print('reply')
                for attr_item in event_item["attributes"]:
                    if attr_item["key"] == '_contract_address':
                        print(f'\treply contract: {get_contract_name(attr_item["value"])}')
                    else:
                        print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'cyberlink':
                print('cyberlinks')
                for i, attr_item in enumerate(event_item['attributes']):
                    if attr_item['key'] == 'particleFrom':
                        print(f'\t{attr_item["value"]} -> {event_item["attributes"][i + 1]["value"]}')
                    elif attr_item['key'] == 'particleTo':
                        pass
                    elif attr_item['key'] == 'neuron':
                        print(f'\tneuron: {get_contract_name(event_item["attributes"][-1]["value"])}\n')
                    else:
                        print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'coin_received':
                print('coin received')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'coin_spent':
                print('coin spent')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'wasm':
                print('wasm')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'transfer':
                print('transfer')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            else:
                print(event_item)

### Create Merkle Root

In [9]:
root_source_list = [{'address': _item['ethereum_address'],
                     'amount': str(_item['amount'])} for _item in claims_list]+ \
                    [{'address': _item['cosmos_address'],
                      'amount': str(_item['amount'])} for _item in claims_list]

with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [10]:
execute_bash(f'rm {ROOT_FILE_NAME}')
root_output, root_error = execute_bash(
    f'merkle-airdrop-cli generateRoot --file {ROOT_SOURCE_FILE_NAME} >> {ROOT_FILE_NAME} && cat {ROOT_FILE_NAME}')
root = root_output.replace('\n', '')
print(f'Merkle root: {root}')

Merkle root: 75ba3c000160ea9239354e82806f1420ddd30b66c7132f4702849e5ae311eb12


In [11]:
root_register_output, root_register_error = \
    execute_bash(
        f'''REGISTER='{{"register_merkle_root":{{"merkle_root":"{root}"}}}}' \
            && CONTRACT="{gift_contract_address}" \
            && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET --broadcast-mode block -y --chain-id={NETWORK} --node={NODE_URL}''')

#### Get Merkle Root form the Gift Contract

In [12]:
print(
    execute_bash(
        f'''QUERY='{{"merkle_root": {{}}}}' \
            && cyber query wasm contract-state smart {gift_contract_address} "$QUERY" --chain-id={NETWORK} --node={NODE_URL}''')[0]
)

data:
  merkle_root: 75ba3c000160ea9239354e82806f1420ddd30b66c7132f4702849e5ae311eb12



### Create Passports

In [13]:
def create_passport(claim_row: pd.Series, display_data: bool = False):
    return execute_bash(
                f'''REGISTER='{{"create_passport":{{"avatar":"{claim_row["avatar"]}", "nickname":"{claim_row["nickname"]}"}}}}' \
                    && CONTRACT="{passport_contract_address}" \
                    && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=600000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 0:
        create_passport_json, _ = create_passport(row, display_data=True)
    else:
        create_passport(row)

0it [00:00, ?it/s]

REGISTER='{"create_passport":{"avatar":"QmdftF8AWKtpqoC3MpNmrapVWiumGTdCPXrvYmz8V17ZVv", "nickname":"john16455994800"}}'                     && CONTRACT="bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra"                     && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=600000 --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443
{
    "code": 0,
    "codespace": "",
    "data": "0A260A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374",
    "gas_used": "400125",
    "gas_wanted": "600000",
    "height": "1426977",
    "info": "",
    "logs": [
        {
            "events": [
                {
                    "attributes": [
                        {
                            "key": "particleFrom",
                            "value": "Qmb6ckJ1zaYx2htqzsgJSK4D7QRbJesPtaL9zYHq5EVnkg"
                        },
                        {
                           

In [14]:
parse_contract_execution_json(create_passport_json)


cyberlinks
	Qmb6ckJ1zaYx2htqzsgJSK4D7QRbJesPtaL9zYHq5EVnkg -> Qmbf497dHw7PB5FwqcSTYzENkwCBSkvACi96hiZXuKcumm
	Qmbf497dHw7PB5FwqcSTYzENkwCBSkvACi96hiZXuKcumm -> Qmb6ckJ1zaYx2htqzsgJSK4D7QRbJesPtaL9zYHq5EVnkg
	neuron: Avatar Subspace Contract

	Qmb6ckJ1zaYx2htqzsgJSK4D7QRbJesPtaL9zYHq5EVnkg -> QmdftF8AWKtpqoC3MpNmrapVWiumGTdCPXrvYmz8V17ZVv
	QmdftF8AWKtpqoC3MpNmrapVWiumGTdCPXrvYmz8V17ZVv -> Qmb6ckJ1zaYx2htqzsgJSK4D7QRbJesPtaL9zYHq5EVnkg
	neuron: Avatar Subspace Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subspace Contract
	execute contract: Avatar Subspace Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Passport Contract
	reply contract: Passport Contract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	token_id: 20


### Proof (Add) Address to Passports

In [15]:
def proof_address(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_bash(
            f'''REGISTER='{{"proof_address":{{"address":"{claim_row[network + "_address"]}", "nickname":"{claim_row["nickname"]}", "signature":"{claim_row[network + "_message_signature"]}"}}}}' \
                && CONTRACT="{passport_contract_address}" \
                && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=400000 --chain-id={NETWORK} --node={NODE_URL}''',
            display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 1:
        proof_ethereum_address_json, _ = proof_address(row, display_data=True)
        proof_cosmos_address_json, _ = proof_address(row, network='cosmos', display_data=True)
    else:
        proof_address(row)
        proof_address(row, network='cosmos', display_data=True)

0it [00:00, ?it/s]

REGISTER='{"proof_address":{"address":"cosmos1za5gkyfz5jp3vhm7lphwgfdvpud87p6mf7xujz", "nickname":"john16455994800", "signature":"eyJwdWJfa2V5IjoiQXBwSVZuMm8yMmlyS1UvR1VjaGdNVHR1cnRUOEJRNEt0bWo0YzhWTFFmbEsiLCJzaWduYXR1cmUiOiI5SHE5Mkl3eXFjZCtMSmsxWWZyNjJPc0txSkhxVVBNTi8wRFNSWEpib3dsK2VhQzVTeXBzZHF3K0NUaFVpY0FxZmpJd05kQ1kybGdIcHh5UEo0bGtQZz09In0="}}'                 && CONTRACT="bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra"                 && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=400000 --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443
{
    "code": 0,
    "codespace": "",
    "data": "0A260A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374",
    "gas_used": "276785",
    "gas_wanted": "400000",
    "height": "1426980",
    "info": "",
    "logs": [
        {
            "events": [
                {
                    "attributes": [
                     

In [16]:
parse_contract_execution_json(proof_ethereum_address_json)


cyberlinks
	Qmb6ckJ1zaYx2htqzsgJSK4D7QRbJesPtaL9zYHq5EVnkg -> QmQx7R3FJKfpmS2eJqaygJRz8y5BYNJbT6rkLbka8bYc4d
	QmQx7R3FJKfpmS2eJqaygJRz8y5BYNJbT6rkLbka8bYc4d -> Qmb6ckJ1zaYx2htqzsgJSK4D7QRbJesPtaL9zYHq5EVnkg
	neuron: Proof Subspace Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subspace Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Passport Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16455994801
	address: 0x9f791bd90b3b3a8f888a1414fba806d7aba85e97


In [17]:
parse_contract_execution_json(proof_cosmos_address_json)


cyberlinks
	Qmb6ckJ1zaYx2htqzsgJSK4D7QRbJesPtaL9zYHq5EVnkg -> Qma4MTYZ9wQoRGUREU8ukrZkqrMVhYdStHKPwbSnDbtGqV
	Qma4MTYZ9wQoRGUREU8ukrZkqrMVhYdStHKPwbSnDbtGqV -> Qmb6ckJ1zaYx2htqzsgJSK4D7QRbJesPtaL9zYHq5EVnkg
	neuron: Proof Subspace Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subspace Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Passport Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16455994801
	address: cosmos120gnxgzj9hv53meq7h24jkgtfw8w2lh47a4ejx


### Generate Proofs and Claim

In [18]:
def generate_proof_and_claim(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    proof_output, _ = execute_bash(f'merkle-airdrop-cli generateProofs --file {ROOT_SOURCE_FILE_NAME} --address {claim_row[network + "_address"]} --amount {claim_row.amount}',
                                   display_data=display_data)
    proof = proof_output.replace('\n','').replace(" ",'').replace("'", '"')
    return execute_bash(
                f'''CLAIM='{{"claim":{{"nickname":"{claim_row['nickname']}", "gift_claiming_address":"{claim_row[network + "_address"]}", "gift_amount":"{claim_row.amount}", "proof":{proof}}}}}' \
                    && cyber tx wasm execute {gift_contract_address} "$CLAIM" --from $WALLET -o json --broadcast-mode block -y --gas 400000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 0:
        claim_ethereum_json, _ = generate_proof_and_claim(row, display_data=True)
        claim_cosmos_json, _ = generate_proof_and_claim(row, network='cosmos', display_data=True)
    else:
        claim_ethereum_json_2, _ = generate_proof_and_claim(row)
        claim_cosmos_json_2, _ = generate_proof_and_claim(row, network='cosmos')

0it [00:00, ?it/s]

merkle-airdrop-cli generateProofs --file root_testing_source_2_addresses.json --address 0x4608e844331df74ca530e0b8cbb80057a8e8818a --amount 10000000
b"[\n  '39203b498080d6acaf9b0973185ff5533bd0bf4609063fac1ea33231d373e19c',\n  'd8338b0fb1d0e60e588200344cde167a9fa1fe6fcd60507811d91c276b335244'\n]\n"
CLAIM='{"claim":{"nickname":"john16455994800", "gift_claiming_address":"0x4608e844331df74ca530e0b8cbb80057a8e8818a", "gift_amount":"10000000", "proof":["39203b498080d6acaf9b0973185ff5533bd0bf4609063fac1ea33231d373e19c","d8338b0fb1d0e60e588200344cde167a9fa1fe6fcd60507811d91c276b335244"]}}'                     && cyber tx wasm execute bostrom1xj68jgjr6u02njplhmkl2f4gg58uueayfrn99x2png87jpsvwx0sk58jjv "$CLAIM" --from $WALLET -o json --broadcast-mode block -y --gas 400000 --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443
{
    "code": 0,
    "codespace": "",
    "data": "0A260A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374",
    "gas_used": "3

In [19]:
parse_contract_execution_json(claim_ethereum_json)


coin received
	receiver: Wallet
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Wallet
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: 0x4608e844331df74ca530e0b8cbb80057a8e8818a
	target: Wallet
	amount: 200000000


In [20]:
parse_contract_execution_json(claim_cosmos_json)


coin received
	receiver: Wallet
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Wallet
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: cosmos1za5gkyfz5jp3vhm7lphwgfdvpud87p6mf7xujz
	target: Wallet
	amount: 200000000


In [22]:
parse_contract_execution_json(claim_ethereum_json_2)


coin received
	receiver: Wallet
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Wallet
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: 0x9f791bd90b3b3a8f888a1414fba806d7aba85e97
	target: Wallet
	amount: 170000000


In [23]:
parse_contract_execution_json(claim_cosmos_json_2)


coin received
	receiver: Wallet
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Wallet
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: cosmos120gnxgzj9hv53meq7h24jkgtfw8w2lh47a4ejx
	target: Wallet
	amount: 140000000


### Release Gift

In [24]:
def release(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_bash(
                f'''CLAIM='{{"release":{{"gift_address":"{claim_row[network + "_address"]}"}}}}' \
                    && cyber tx wasm execute {gift_contract_address} "$CLAIM" --from $WALLET -o json --broadcast-mode block -y --gas 400000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 0:
        release_ethereum_json, _ = release(row, display_data=True)
        release_cosmos_json, _ = release(row, network='cosmos', display_data=True)
    else:
        release(row)
        release(row, network='cosmos')


0it [00:00, ?it/s]

CLAIM='{"release":{"gift_address":"0x4608e844331df74ca530e0b8cbb80057a8e8818a"}}'                     && cyber tx wasm execute bostrom1xj68jgjr6u02njplhmkl2f4gg58uueayfrn99x2png87jpsvwx0sk58jjv "$CLAIM" --from $WALLET -o json --broadcast-mode block -y --gas 400000 --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443
{
    "code": 0,
    "codespace": "",
    "data": "0A260A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374",
    "gas_used": "158902",
    "gas_wanted": "400000",
    "height": "1426997",
    "info": "",
    "logs": [
        {
            "events": [
                {
                    "attributes": [
                        {
                            "key": "receiver",
                            "value": "bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz"
                        },
                        {
                            "key": "amount",
                            "value": "19990000boot"
                    

In [25]:
parse_contract_execution_json(release_ethereum_json)


coin received
	receiver: Wallet
	amount: 19990000boot

coin spent
	spender: Gift Contract
	amount: 19990000boot

execute
	execute contract: Gift Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Wallet
	sender: Gift Contract
	amount: 19990000boot

wasm
	_contract_address: Gift Contract
	action: release
	address: Wallet
	gift_address: 0x4608e844331df74ca530e0b8cbb80057a8e8818a
	stage: 9
	amount: 19990000


In [26]:
parse_contract_execution_json(release_cosmos_json)


coin received
	receiver: Wallet
	amount: 19990000boot

coin spent
	spender: Gift Contract
	amount: 19990000boot

execute
	execute contract: Gift Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Wallet
	sender: Gift Contract
	amount: 19990000boot

wasm
	_contract_address: Gift Contract
	action: release
	address: Wallet
	gift_address: cosmos1za5gkyfz5jp3vhm7lphwgfdvpud87p6mf7xujz
	stage: 9
	amount: 19990000


### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subspaces
#### transfer_nft
#### send_nft
#### mint
#### burn
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for Subspace contract
#### update_owner
#### update_executer +